In [1]:
!pip install surprise

     |████████████████████████████████| 11.8 MB 4.1 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619417 sha256=56769fa8462d5e88d165e3ea58e0c88d29910cc99398702ec0651cdad9cd1cda
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
import pandas as pd

In [4]:
# about rating
path_rating = 'drive/MyDrive/Notebooks/ml-100k/u.data'
# about movie
path_movie = 'drive/MyDrive/Notebooks/ml-100k/u.item'
# about user
path_user = 'drive/MyDrive/Notebooks/ml-100k/u.user'

In [5]:
def parse(s):
  seq = s.split(' | ')
  return list(map(lambda s: s.replace(' ', '_'), seq))

parse('user id | age | gender | occupation | zip code')

['user_id', 'age', 'gender', 'occupation', 'zip_code']

In [10]:
# user id | item id | rating | timestamp
rating_col = parse('user id | movie id | rating | timestamp')
print(rating_col)
ratings = pd.read_csv(path_rating, sep='\t', names=rating_col, encoding='ISO-8859-1')
print(ratings.shape)
display(ratings.head(5))

['user_id', 'movie_id', 'rating', 'timestamp']
(100000, 4)


,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [7]:
from surprise import Dataset
from surprise import Reader

In [17]:
# 데이터 생성 (convert to explicit data for analysis in surprise module)
reader = Reader(rating_scale=(ratings['rating'].min(), ratings['rating'].max()))  # reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[["user_id", "movie_id", "rating"]], reader)
display(data)

# 학습 / 데이터 생성 
train_data = data.build_full_trainset()
test_data = train_data.build_testset()

In [18]:
from surprise import SVD

algo = SVD(n_factors=50, n_epochs=50)
algo.fit(train_data)

In [19]:
prediction = algo.test(test_data)
print(prediction[:3])
# uid : user id, iid : item id, r_ui : rating값, est : 예측값

[Prediction(uid=196, iid=242, r_ui=3.0, est=3.442356680431108, details={'was_impossible': False}), Prediction(uid=196, iid=393, r_ui=4.0, est=3.972203659837599, details={'was_impossible': False}), Prediction(uid=196, iid=381, r_ui=4.0, est=3.639906501268671, details={'was_impossible': False})]


In [20]:
from surprise import accuracy

accuracy.rmse(prediction)

RMSE: 0.5049


0.5048635843481354

In [24]:
from surprise.model_selection import GridSearchCV

# lr_all : learning rate, reg_all : regularization rate
param = {"n_factors": [50, 100], "lr_all": [0.01, 0.02], "reg_all": [0.01, 0.02]}
gs = GridSearchCV(algo_class=SVD, measures=["rmse"], param_grid=param)
gs.fit(data)

print(gs.best_params["rmse"])

{'n_factors': 50, 'lr_all': 0.01, 'reg_all': 0.02}


In [25]:
algo = SVD(n_factors=50, n_epochs=50, lr_all=0.01, reg_all=0.02)
algo.fit(train_data)
prediction = algo.test(test_data)
accuracy.rmse(prediction)

RMSE: 0.3942


0.3941793335757115